# Intervalo de verosimilitud relativa de $\lambda$ para una muestra de variables aleatorias exponenciales

Para la solución del problema se hace uso del método de Newton-Raphson, cuyo implementación se muestra a continuación.

## Líbrerias y clases que se usaran

In [1]:
import numpy as np
import plotly.graph_objects as go

## Intervalo de verosimilitud

Recibe como parámetros una función de verosimilitud $L$, el porcentaje $p$, el $emv$ de $L$, la cantidad de pasos para calcularlo $iterMax$ y la tolerancia de error $tol$. Imprime el intervalo de verosimilitud del $p\%$.

In [2]:
from scipy import optimize

def likelihood_profile_interval_mountain(R, p, emv, param):
    
    f = lambda x, param : R(x,param) - p
    roots = optimize.root_scalar(f,args = param, bracket = [0.00001,emv-0.01]), optimize.root_scalar(f,args = param, bracket = [emv+0.01,5])
    return roots[0].root, roots[1].root
    

In [3]:
def R(x,param):
    return 2**(2*param[0]) * (param[2]/(x*param[1]))**(param[0]) * (1 + (param[2]/(x*param[1])))**(-2*param[0])

## Código Prinicipal

Declaración de parametros y funciones para estimar $\hat{\lambda}$, frecuencias esperadas e intervalo de verosimilitud $p$.

In [6]:
from pprint import pprint

def c_1(frec):
    return sum((frec[0][j]+1)*frec[1][j] for j in range(len(frec[0])))
def c_2(frec):
    return sum(frec[1][j] for j in range(len(frec[0])))
    
def main():
    
    iterMax = int(1e3)
    tol = 1e-6
    n = 6
    x = [9,186,25,6,44,115]
    y = [1,18,6,25,14,45]
    p = 0.10
    param = [n,sum(x)/n,sum(y)/n]
    emv_lambda = lambda x,y : sum(y)/sum(x)
    tl, tr = likelihood_profile_interval_mountain(R, p, emv_lambda(x,y), param)
    z = np.linspace(0+tol,4,iterMax)
    z = np.concatenate((z, np.array([tl,tr,emv_lambda(x,y)])))
    z = np.sort(z)
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=z,y=[R(x,param) for x in z],
            mode="lines", name=r'$R(\lambda$)'
        )
    )
    z = [tl,tr,emv_lambda(x,y)]
    fig.add_trace(
        go.Scatter(
            x=z,
            y=[R(x,param) for x in z],
            mode="markers",
            marker = dict(size=10),
            name='Emv'
        )
    )
    z = [tl,tr]
    fig.add_trace(
        go.Scatter(
            x=z,
            y=[R(x,param) for x in z],
            mode="markers",
            marker = dict(size=10),
            name=fr'$R(\lambda) = {p}$'
        )
    )
    z = np.linspace(tl,tr,iterMax)
    fig.add_trace(
        go.Scatter(
            x=z,y=[p for _ in z],
            mode="lines"
        )
    )
    fig.update_yaxes(zeroline=True)
    fig.update_layout(
        template = "simple_white",
        title = dict(
            text = r"Verosimilitud perfil relativa de $\lambda$",
            x = 0.5,
            xanchor = "center"
        ),
        hovermode="x unified"
    )
    fig.show()
    print("El intervalo de verosimilitud del {}% es: [{},{}]".format(p*100, round(tl,4), round(tr,4)))


In [7]:
main()

El intervalo de verosimilitud del 10.0% es: [0.0788,1.0172]
